# Publishing Notebooks and Notebook-based Tools

## What is Publishing?


Publishing a notebook is similar to publishing anything on the hub platform. By publishing your work you will:

* Make your work available to others.
* Allow others to cite your work. 

To do this, you will have to go to https://nanohub.org/tools/create and follow the process described there.  Detailed notes follow.

## Notebooks vs Notebook-based Tools

Publishing a notebook is just making your notebook available for others to run.  They will have to step through the cells (or select "Cell/Run All") manually.  This is great for tutorials and workflows, but my be distracting for tools where you don't want the users
to see (or optionally modify) the code cells.  

A "Notebook-based Tool" is a notebook that runs automatically.  Code cells are hidden. You may insert waitpoints which require the user to click a button to proceed.  You can include interactive widgets to get inputs or visualize the outputs.  Using this mode, it is possible to make tools similar to the Rappture-based tools.

## Publishing Notebooks

A PDF containing the process for [Publishing Notebooks](PublishingNotebooks.pdf). 

## Publishing Notebook-based Tools

To describe the steps to publish a notebook as a tool, we will create and publish a new notebook tool, based on https://nanohub.org/resources/gptool.


### 1. First we go to https://nanohub.org/tools/create and create a new tool, https://nanohub.org/tools/gptool2.  

### 2. Then open a terminal from the notebook home or dashboard view.  You will be in your notebooks directory.

```bash
~/notebooks> svn checkout https://nanohub.org/tools/gptool2/svn/trunk gptool2                                                      
A    gptool2/middleware                                                                                                            
A    gptool2/middleware/invoke                                                                                                     
A    gptool2/examples                                                                                                              
A    gptool2/rappture                                                                                                              
A    gptool2/doc                                                                                                                   
A    gptool2/src                                                                                                                   
A    gptool2/src/Makefile                                                                                                          
A    gptool2/bin                                                                                                                   
A    gptool2/data                                                                                                                  
Checked out revision 1.
```              

### 3.  Go back to the home(dashboard) page and go into the gptool2/bin folder.  Create a new notebook there.

Optionally use the upload button to upload a notebook you have already started.  We also uploaded some python code
("gpr.py) and some test data.

### 4. Get the notebook working.

You may want to use notebook UI elements from http://hublib.readthedocs.io/en/latest/intro.html
You can use ***%use*** from hublib to load any required environment modules.



[This](publishing/Gaussian1.ipynb) is the initial version.

### 5. Insert Waitpoints

Tool-mode notebooks automatically start running when the notebook is loaded. If there are places where you would like execution to pause, you
will need to insert waitpoints.  Execution pauses at waitpoints and will only resume when the user clicks the continue button. You can read more about them at http://hublib.readthedocs.io/en/latest/tools.html

In our example, we will insert a single waitpoint after displaying the file upload dialog, allowing the user to upload the file before continuing.  Waitpoint cells have a green background when execution is paused on them and a red background when the next cells are running.  We put a "%waitdone" at the end of the notebook, which simply displays "DONE" and turns off the red background.


[This](publishing/Gaussian2.ipynb) is the new version.

### 6. Activate Tool Mode and Test

Notebooks will only run in tool-mode if tool-mode is enabled and the notebook is signed as "trusted".  Tool-mode only needs to be enabled once, but every time the notebook sources change, it must be signed as trusted.


To set tool-mode, you will need to go back to the terminal. First we set tool mode then mark it trusted.
```    
~/notebooks/gptool2/bin> jupyter_tool.py -t Gaussian.ipynb                                                                         
Tool Mode is True                                                                                                                  
~/notebooks/gptool2/bin> jupyter trust Gaussian.ipynb                                                                              
Signing notebook: Gaussian.ipynb                                                                                                   
~/notebooks/gptool2/bin>
```

Next time you run the notebook, it should be running in tool mode.

You cannot make any changes while in tool mode, so if you need to modify the notebook, you will have to run it as a normal notebook.  The easiest way to do this is to "unsign" it so it will no longer be trusted.

```
~/notebooks/gptool2/bin> jupyter_unsign.py Gaussian.ipynb                                                                          
```

When you are finished with your modifications, just mark it as trusted again:
```
~/notebooks/gptool2/bin> jupyter trust Gaussian.ipynb                                                                              
Signing notebook: Gaussian.ipynb 
```


              

### 7. Write your Invoke script

When you created your tool, the tool creation process automatically put something like the following in middleware/invoke.

``` bash
~/notebooks/gptool2/middleware> cat invoke                                                                                         
#!/bin/sh                                                                                                                          
                                                                                                                                   
/usr/bin/invoke_app "$@" -C rappture -t gptool2      
```

You should change it to something like:

```
#!/bin/sh
/usr/bin/invoke_app -C "start_jupyter -t -T @tool bin/Gaussian.ipynb" -u anaconda2-4.1                                          
```

You should try using the editor built into Jupyter.  From the dashboard, just click on the 'middleware' folder, then click on the 'invoke' file.

All notebooks are started by 'start_jupyter" which is in the anaconda2-4.1 module.  You should always have "-u anaconda2-4.1" in your invoke script regardless of what version of python you are using.

"-t" enables tool mode.  

"-T @tool" is always required.

"bin/Gaussian.ipynb" is the notebook to start.  Path is relative to the tool directory.



### 8. Check in Your Changes

``` bash
~/notebooks/gptool2> svn status                                                                                                    
?       bin/test.dat                                                                                                               
?       bin/.ipynb_checkpoints                                                                                                     
?       bin/Gaussian.ipynb                                                                                                         
?       bin/gpr.py                                                                                                                 
?       bin/TESTDAT.DAT                                                                                                            
M       middleware/invoke                                                                                                          
~/notebooks/gptool2> svn add bin/Gaussian.ipynb  bin/gpr.py bin/test.dat                                                           
A         bin/Gaussian.ipynb                                                                                                       
A         bin/gpr.py                                                                                                               
A         bin/test.dat                                                                                                             
~/notebooks/gptool2> svn commit -m 'initial version'                                                                               
Authentication realm: <https://nanohub.org:443> Notebook: Gaussian process regression in 1D Subversion Repository                  
Password for 'mmh':                                                                                                                
Adding         bin/Gaussian.ipynb                                                                                                  
Adding         bin/gpr.py                                                                                                          
Adding         bin/test.dat                                                                                                        
Sending        middleware/invoke                                                                                                   
Transmitting file data ....                                                                                                        
Committed revision 2.                                                                                                              

```


### 9. Update Tool Status

Go to your tool status page (for example
https://nanohub.org/tools/gptool/status)

and click on "I've made changes Please install the latest code for testing and approval"
